Import:

In [126]:
import numpy as np
from numpy import unique
from numpy import argmax
import math
from pandas import read_csv
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from pandas import read_csv
from matplotlib import pyplot
from sklearn import preprocessing

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

from tensorflow.python.client import device_lib 
CUDA_VISIBLE_DEVICES = 0

Num GPUs Available:  1


Data Import:

In [128]:
def cutoff(pred, real, score, percent):
    sampleNum, correct, cutoff = len(real), 1, 0.0001

    while (sampleNum/len(real))>percent/100:
        sampleNum, correct = 0,0
        for i in range(len(pred)):
            if score[i,argmax(score[i])]>cutoff:
                sampleNum+=1
                if real[i]==pred[i]:
                    correct+=1
        cutoff += 0.0001
        
    if sampleNum and correct !=0:
        print(percent, "Percent Test: ",sampleNum, correct, correct/sampleNum*100, "%, Cutoff Value:", cutoff)
    else:
        print("NaN")
        
def ones(pred, real, score):
    sampleNum, correct = 0, 0

    for i in range(len(pred)):
        if score[i,argmax(score[i])]>=1:
            sampleNum+=1
            if real[i]==pred[i]:
                correct+=1
        
    if sampleNum and correct !=0:
        print("Ones: ",sampleNum, correct, correct/sampleNum*100, "%, Cutoff Value:")
    else:
        print("NaN")
    
def softmaxTest (yhat, y_test, probs):
    sampleNum, correct = 0,0
    print("Normal Test")
    for i in range(len(y_test)):
        sampleNum+=1
        if y_test[i]==yhat[i]:
            correct+=1

    if sampleNum and correct !=0:
         print(sampleNum, correct, correct/sampleNum*100, "%")
    else:
         print("NaN")
    
    print("Cutoff Softmax Test")
    cutoff(yhat, y_test, probs, 90)
    cutoff(yhat, y_test, probs, 75)
    cutoff(yhat, y_test, probs, 66.6667)
    cutoff(yhat, y_test, probs, 50)
    cutoff(yhat, y_test, probs, 33.3333)
    cutoff(yhat, y_test, probs, 20)
    ones(yhat, y_test, probs)
           
    
    

    true, false = 0, 0
    tCounter, fCounter = 0, 0
    for i in range(len(probs)):
        if(yhat[i]!=y_test[i]):
            false+=probs[i,argmax(probs[i])]
            fCounter+=1
            #print(probs[i,argmax(probs[i])],argmax(probs[i]), yhat[i],y_test[i])
        else:
            true+=probs[i,argmax(probs[i])]
            tCounter+=1

    print("True data average softmax value: ", true/tCounter)
    print("False data average softmax value: ", false/fCounter)


In [201]:
def dataPipe3(path):
    dataframe = read_csv(path, header=0)
    pitches = dataframe.iloc[0:,0]
    pitches_List = ["Fastball", "Breaking Ball", "Offspeed", "Other"]
    counter = 0

    dl_Length = len(pitches)
    num_Pitches = len(pitches_List)
    
    dataset = np.zeros([dl_Length,num_Pitches*8+23])
    x_Length,y_Length = np.shape(dataset)
    for i in range(len(pitches)):
        if pitches[i] == "FF" or  pitches[i] == "FT" or pitches[i] == "FS" or pitches[i] ==  "FC" or pitches[i] ==  "SI": # fastball
            dataset[i,y_Length-1] = 0
        elif pitches[i] == "CU" or pitches[i] ==  "SL" or pitches[i] == "KC" or pitches[i] == "EP": # breaking ball
            dataset[i,y_Length-1] = 1
        elif pitches[i] == "CH" or pitches[i] == "SF" or pitches[i] == "SC" or pitches[i] == "FO" or pitches[i] == "KN": # offspeed
            dataset[i,y_Length-1] = 2
        else:
            dataset[i,y_Length-1] = 3 
                
    for i in range(dl_Length):
        # date
        dataset[i,0] = int(str(dataframe.iloc[i,1]) .replace("-",""))
        # batter ID
        dataset[i,1] = int(dataframe.iloc[i,6])
        #pitcher ID
        dataset[i,2] = int(dataframe.iloc[i,7])


        # R is 0 and L is one
        if(dataframe.iloc[i,16] == "R"):
            dataset[i,3] = 0
        elif(dataframe.iloc[i,16] == "L"):
            dataset[i,3] = 1

        # R is 0 and L is one
        if(dataframe.iloc[i,17] == "R"):
            dataset[i,4] = 0
        elif(dataframe.iloc[i,17] == "L"):
            dataset[i,4] = 1

        # R is 0 and L is one
        if(dataframe.iloc[i,18] == "R"):
            dataset[i,5] = 0
        elif(dataframe.iloc[i,18] == "L"):
            dataset[i,5] = 1

        #Ball  
        dataset[i,6] = int(dataframe.iloc[i,24])
        #Strike
        dataset[i,7] = int(dataframe.iloc[i,25])

        # On base
        # 3B
        if(math.isnan(dataframe.iloc[i,31])):

            dataset[i,8] = 0
        else:
            dataset[i,8] = 1


        #2B   
        if(math.isnan(dataframe.iloc[i,31])):
            dataset[i,9] = 0
        else:
            dataset[i,9] = 1
        #1B
        if(math.isnan(dataframe.iloc[i,31])):
            dataset[i,10] = 0
        else:
            dataset[i,10] = 1

        # innings
        dataset[i,11] = int(dataframe.iloc[i,35])

        # outs
        dataset[i,12] = int(dataframe.iloc[i,34])

        # Catcher
        if(math.isnan(dataframe.iloc[i,60])):
            dataset[i,13] = 0
        else:
            dataset[i,13] = int(dataframe.iloc[i,60])

        # Score
        #bat score
        dataset[i,14] = int(dataframe.iloc[i,81])
        #Field Score
        dataset[i,15] = int(dataframe.iloc[i,82])

        # Year/Career Pitches number
        dataset[i,16] = dl_Length-1-i

        dataset[dl_Length-1,17] = 0
        dataset[dl_Length-1,18] = 0
        dataset[dl_Length-1,19] = 0
        dataset[dl_Length-1,20:y_Length-4]=0
        dataset[dl_Length-1,y_Length-3] = -1
        dataset[dl_Length-1, y_Length-2] = 1
        dataset[dl_Length-2, y_Length-2] = 1

    # game, out and inning pitch count 
    for i in range(dl_Length-2,-1,-1):
        # streak type
        dataset[i,y_Length-3] = dataset[i+1,y_Length-1]
        # game
        if dataset[i,0]!=dataset[i+1,0]:
            dataset[i,17] = 0
        else:
            dataset[i,17] = dataset[i+1,17]+1

        # inning
        if dataset[i,11]!=dataset[i+1,11]:
            dataset[i,18] = 0
        else:
            dataset[i,18] = dataset[i+1,18]+1

        # out 
        if dataset[i,1]!=dataset[i+1,1]:
            dataset[i,19] = 0
        else:
            dataset[i,19] = dataset[i+1,19]+1

        # year/career pitch tracker
        dataset[i,20:20+num_Pitches] = dataset[i+1,20:20+num_Pitches]
        dataset[i,20+int(dataset[i+1,y_Length-1])]+=1

        # percentages
        if dataset[i,16]==0:
            dataset[i,20+num_Pitches:20+num_Pitches*2] = 0
        else:
            dataset[i,20+num_Pitches:20+num_Pitches*2] = dataset[i,20:20+num_Pitches]/dataset[i,16]

        #game pitch tracker
        if dataset[i+1,0]!=dataset[i,0]:
            dataset[i,20+num_Pitches*2:20+num_Pitches*3] = 0
        else:
            dataset[i,20+num_Pitches*2:20+num_Pitches*3] = dataset[i+1,20+num_Pitches*2:20+num_Pitches*3]
            dataset[i,20+num_Pitches*2+int(dataset[i+1,y_Length-1])]+=1

         # percentages
        if dataset[i,17]==0:
            dataset[i,20+num_Pitches*3:20+num_Pitches*4] = 0
        else:
            dataset[i,20+num_Pitches*3:20+num_Pitches*4] = dataset[i,20+num_Pitches*2:20+num_Pitches*3]/dataset[i,17]

        #inning pitch tracker
        if dataset[i+1,11]!=dataset[i,11]:
            dataset[i,20+num_Pitches*4:20+num_Pitches*5] = 0
        else:
            dataset[i,20+num_Pitches*4:20+num_Pitches*5] = dataset[i+1,20+num_Pitches*4:20+num_Pitches*5]
            dataset[i,20+num_Pitches*4+int(dataset[i+1,y_Length-1])]+=1

         # percentages
        if dataset[i,18]==0:
            dataset[i,20+num_Pitches*5:20+num_Pitches*6] = 0
        else:
            dataset[i,20+num_Pitches*5:20+num_Pitches*6] = dataset[i,20+num_Pitches*4:20+num_Pitches*5]/dataset[i,18]


        #out pitch tracker

        if dataset[i+1,1]!=dataset[i,1]:
            dataset[i,20+num_Pitches*6:20+num_Pitches*7] = 0
        else:
            dataset[i,20+num_Pitches*6:20+num_Pitches*7] = dataset[i+1,20+num_Pitches*6:20+num_Pitches*7]
            dataset[i,20+num_Pitches*6+int(dataset[i+1,y_Length-1])]+=1

         # percentages
        if dataset[i,19]==0:
            dataset[i,20+num_Pitches*7:20+num_Pitches*8] = 0
        else:
            dataset[i,20+num_Pitches*7:20+num_Pitches*8] = dataset[i,20+num_Pitches*6:20+num_Pitches*7]/dataset[i,19]  
            
    # streak length
    for i in range(dl_Length-3,-1,-1):
        if dataset[i,y_Length-1] == dataset[i+1,y_Length-1]:
            dataset[i,y_Length-2] = dataset[i+1,y_Length-2] +1
    
    np.savetxt("C:\\Users\\16479\\Desktop\\foo.csv", dataset, delimiter=",")
    
    return dataset

def dataPipe(path):
    dataframe = read_csv(path, header=0)
    pitches = dataframe.iloc[0:,0]
    pitches_List = []
    counter = 0

    dl_Length = len(pitches)

    for i in range(len(pitches)):
        exists = False
        j = 0 
        for j in range(len(pitches_List)):
            if pitches[i] == pitches_List[j]:
                exists = True
        if exists == False:
            pitches_List.append(pitches[i])
            counter+=1


    num_Pitches = len(pitches_List)

    dataset = np.zeros([dl_Length,num_Pitches*8+23])
    x_Length,y_Length = np.shape(dataset)
    for i in range(len(pitches)):
        for j in range(len(pitches_List)):
             if pitches[i] == pitches_List[j]:
                dataset[i,y_Length-1] = j 
    for i in range(dl_Length):
        # date
        dataset[i,0] = int(str(dataframe.iloc[i,1]) .replace("-",""))
        # batter ID
        dataset[i,1] = int(dataframe.iloc[i,6])
        #pitcher ID
        dataset[i,2] = int(dataframe.iloc[i,7])


        # R is 0 and L is one
        if(dataframe.iloc[i,16] == "R"):
            dataset[i,3] = 0
        elif(dataframe.iloc[i,16] == "L"):
            dataset[i,3] = 1

        # R is 0 and L is one
        if(dataframe.iloc[i,17] == "R"):
            dataset[i,4] = 0
        elif(dataframe.iloc[i,17] == "L"):
            dataset[i,4] = 1

        # R is 0 and L is one
        if(dataframe.iloc[i,18] == "R"):
            dataset[i,5] = 0
        elif(dataframe.iloc[i,18] == "L"):
            dataset[i,5] = 1

        #Ball  
        dataset[i,6] = int(dataframe.iloc[i,24])
        #Strike
        dataset[i,7] = int(dataframe.iloc[i,25])

        # On base
        # 3B
        if(math.isnan(dataframe.iloc[i,31])):

            dataset[i,8] = 0
        else:
            dataset[i,8] = 1


        #2B   
        if(math.isnan(dataframe.iloc[i,31])):
            dataset[i,9] = 0
        else:
            dataset[i,9] = 1
        #1B
        if(math.isnan(dataframe.iloc[i,31])):
            dataset[i,10] = 0
        else:
            dataset[i,10] = 1

        # innings
        dataset[i,11] = int(dataframe.iloc[i,35])

        # outs
        dataset[i,12] = int(dataframe.iloc[i,34])

        # Catcher
        if(math.isnan(dataframe.iloc[i,60])):
            dataset[i,13] = 0
        else:
            dataset[i,13] = int(dataframe.iloc[i,60])

        # Score
        #bat score
        dataset[i,14] = int(dataframe.iloc[i,81])
        #Field Score
        dataset[i,15] = int(dataframe.iloc[i,82])

        # Year/Career Pitches number
        dataset[i,16] = dl_Length-1-i

        dataset[dl_Length-1,17] = 0
        dataset[dl_Length-1,18] = 0
        dataset[dl_Length-1,19] = 0
        dataset[dl_Length-1,20:y_Length-4]=0
        dataset[dl_Length-1,y_Length-3] = -1
        dataset[dl_Length-1, y_Length-2] = 1
        dataset[dl_Length-2, y_Length-2] = 1

    # game, out and inning pitch count 
    for i in range(dl_Length-2,-1,-1):
        # streak type
        dataset[i,y_Length-3] = dataset[i+1,y_Length-1]
        # game
        if dataset[i,0]!=dataset[i+1,0]:
            dataset[i,17] = 0
        else:
            dataset[i,17] = dataset[i+1,17]+1

        # inning
        if dataset[i,11]!=dataset[i+1,11]:
            dataset[i,18] = 0
        else:
            dataset[i,18] = dataset[i+1,18]+1

        # out 
        if dataset[i,1]!=dataset[i+1,1]:
            dataset[i,19] = 0
        else:
            dataset[i,19] = dataset[i+1,19]+1

        # year/career pitch tracker
        dataset[i,20:20+num_Pitches] = dataset[i+1,20:20+num_Pitches]
        dataset[i,20+int(dataset[i+1,y_Length-1])]+=1

        # percentages
        if dataset[i,16]==0:
            dataset[i,20+num_Pitches:20+num_Pitches*2] = 0
        else:
            dataset[i,20+num_Pitches:20+num_Pitches*2] = dataset[i,20:20+num_Pitches]/dataset[i,16]

        #game pitch tracker
        if dataset[i+1,0]!=dataset[i,0]:
            dataset[i,20+num_Pitches*2:20+num_Pitches*3] = 0
        else:
            dataset[i,20+num_Pitches*2:20+num_Pitches*3] = dataset[i+1,20+num_Pitches*2:20+num_Pitches*3]
            dataset[i,20+num_Pitches*2+int(dataset[i+1,y_Length-1])]+=1

         # percentages
        if dataset[i,17]==0:
            dataset[i,20+num_Pitches*3:20+num_Pitches*4] = 0
        else:
            dataset[i,20+num_Pitches*3:20+num_Pitches*4] = dataset[i,20+num_Pitches*2:20+num_Pitches*3]/dataset[i,17]

        #inning pitch tracker
        if dataset[i+1,11]!=dataset[i,11]:
            dataset[i,20+num_Pitches*4:20+num_Pitches*5] = 0
        else:
            dataset[i,20+num_Pitches*4:20+num_Pitches*5] = dataset[i+1,20+num_Pitches*4:20+num_Pitches*5]
            dataset[i,20+num_Pitches*4+int(dataset[i+1,y_Length-1])]+=1

         # percentages
        if dataset[i,18]==0:
            dataset[i,20+num_Pitches*5:20+num_Pitches*6] = 0
        else:
            dataset[i,20+num_Pitches*5:20+num_Pitches*6] = dataset[i,20+num_Pitches*4:20+num_Pitches*5]/dataset[i,18]


        #out pitch tracker

        if dataset[i+1,1]!=dataset[i,1]:
            dataset[i,20+num_Pitches*6:20+num_Pitches*7] = 0
        else:
            dataset[i,20+num_Pitches*6:20+num_Pitches*7] = dataset[i+1,20+num_Pitches*6:20+num_Pitches*7]
            dataset[i,20+num_Pitches*6+int(dataset[i+1,y_Length-1])]+=1

         # percentages
        if dataset[i,19]==0:
            dataset[i,20+num_Pitches*7:20+num_Pitches*8] = 0
        else:
            dataset[i,20+num_Pitches*7:20+num_Pitches*8] = dataset[i,20+num_Pitches*6:20+num_Pitches*7]/dataset[i,19]  
            
    # streak length
    for i in range(dl_Length-3,-1,-1):
        if dataset[i,y_Length-1] == dataset[i+1,y_Length-1]:
            dataset[i,y_Length-2] = dataset[i+1,y_Length-2] +1
    
    return dataset

# split data into train and test sets


def modelRun(data, e, bs, v):
    #np.savetxt("C:\\Users\\16479\\Desktop\\foo.csv", dataset, delimiter=",")
    # split into input (X) and output (y) variables
    #dataframe = read_csv("C:\\Users\\16479\\Desktop\\Matz_Audible.csv", header=0)
    #dataset = dataframe.values
    data_X, data_y = data[:, 1:-1], data[:, -1]
    data_X, data_y = data_X.astype('float'), data_y.astype('float')
    n_features = data_X.shape[1]
    data_X = preprocessing.scale(data_X)
    #X_train, X_test, y_train, y_test = data_X[113:], data_X[0:113], data_y[113:], data_y[0:113]
    X_train, X_test, y_train, y_test =train_test_split(data_X, data_y, test_size=0.2, random_state=2)

    data_y = LabelEncoder().fit_transform(data_y)
    n_class = len(unique(data_y))
    
    configuration = tf.compat.v1.ConfigProto()
    configuration.gpu_options.allow_growth = True
    session = tf.compat.v1.Session(config=configuration)

    model = Sequential()
    model.add(Dense(90, input_dim=n_features, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(70, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(20, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(n_class, activation='softmax'))


    model.compile(loss='sparse_categorical_crossentropy', optimizer="RMSprop")
    # fit the keras model on the dataset
    #Optimizers: adam, SGD, RMSprop, adagard
    #loss: sparse categorical crossentropy,
    #different batch sizes

    history=model.fit(X_train, y_train, epochs = e, batch_size = bs, verbose = v)
 
    # evaluate on test set
    yhat = model.predict(X_test)
    probs = model.predict_proba(X_test)

    yhat = argmax(yhat, axis=-1).astype('int')
    acc = accuracy_score(y_test, yhat)
    return acc, yhat, y_test, probs, model
    #print('Accuracy: %.3f' % acc)
    
def modelRunTest(data, test, e, bs, v):
    data_X, data_y = data[:, 1:-1], data[:, -1]
    data_X, data_y = data_X.astype('float'), data_y.astype('float')
    
    data_XT, data_yT = test[:, 1:-1], test[:, -1]
    data_XT, data_yT = data_XT.astype('float'), data_yT.astype('float')
    
    n_features = data_X.shape[1]
    data_X = preprocessing.scale(data_X)
    data_XT = preprocessing.scale(data_XT)
    X_train, X_test, y_train, y_test = data_X, data_XT, data_y, data_yT

    data_y = LabelEncoder().fit_transform(data_y)
    n_class = len(unique(data_y))
    
    configuration = tf.compat.v1.ConfigProto()
    configuration.gpu_options.allow_growth = True
    session = tf.compat.v1.Session(config=configuration)

    model = Sequential()
    model.add(Dense(90, input_dim=n_features, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(70, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(20, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(n_class, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy', optimizer="RMSprop")
    # fit the keras model on the dataset
    #Optimizers: adam, SGD, RMSprop, adagard
    #loss: sparse categorical crossentropy,
    #different batch sizes

    history=model.fit(X_train, y_train, epochs = e, batch_size = bs, verbose = v)
 
    # evaluate on test set
    yhat = model.predict(X_test)
    probs = model.predict_proba(X_test)

    yhat = argmax(yhat, axis=-1).astype('int')
    acc = accuracy_score(y_test, yhat) 
    return acc, yhat, y_test, probs, model

def modelRunTestSingle(data, test, e, bs, v):
    x,y = np.shape(test)
    data_X, data_y = data[:, 1:-1], data[:, -1]
    data_X, data_y = data_X.astype('float'), data_y.astype('float')
    
    data_XT, data_yT = test[:, 1:y-1], test[:, y-1]
    data_XT, data_yT = data_XT.astype('float'), data_yT.astype('float')
    
    n_features = data_X.shape[1]
    data_X = preprocessing.scale(data_X)
    data_XT = preprocessing.scale(data_XT)
    X_train, X_test, y_train, y_test = data_X, data_XT, data_y, data_yT

    data_y = LabelEncoder().fit_transform(data_y)
    n_class = len(unique(data_y))
    
    configuration = tf.compat.v1.ConfigProto()
    configuration.gpu_options.allow_growth = True
    session = tf.compat.v1.Session(config=configuration)

    model = Sequential()
    model.add(Dense(90, input_dim=n_features, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(70, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(20, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(n_class, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy', optimizer="RMSprop")
    # fit the keras model on the dataset
    #Optimizers: adam, SGD, RMSprop, adagard
    #loss: sparse categorical crossentropy,
    #different batch sizes

    history=model.fit(X_train, y_train, epochs = e, batch_size = bs, verbose = v)
 
    # evaluate on test set
    yhat = model.predict(X_test)
    probs = model.predict_proba(X_test)

    yhat = argmax(yhat, axis=-1).astype('int')
    acc = accuracy_score(y_test, yhat) 
    return acc, yhat, y_test, probs, model




def modelRunUpdateTest(data, test, epoch, batch, verb):# finish this
    length, width = np.shape(test)
    guesses = np.zeros((length))
    pitches = np.zeros((length))
    for i in range(len(test)):
        tData = np.zeros((1,width))
        tData[:,0:width] = test[length-i-1]
        
        
        acc, preds, real, probs, model = modelRunTestSingle(data, tData, epoch, batch, verb)
        data = np.append(data, tData, axis = 0)
        guesses[i] = preds
        pitches[i] = real


    correct, total = 0, 0
    for i in range(len(guesses)):
        if guesses[i]==pitches[i]:
            correct+=1
        total+=1
    print(correct/total, " %")
    return acc, guesses, pitches


In [215]:
one, two, three = modelRunUpdateTest(d3a, t3, 20, 32, 0)

0.46875  %


In [ ]:
t1 = d1[0:69]
d1a =  d1[69:]
fd1 = d1a
dSet1 = [d1a, d2, d3, d4, d5, d6, d7, d8, d9, d10, d11]

for i in range(1,11):
    fd1 = np.concatenate((fd1, dSet1[i]), axis=0)

t2 = d2[0:112]
d2a =  d2[112:]
fd2 = d2a
dSet2 = [d2a, d1, d3, d4, d5, d6, d7, d8, d9, d10, d11]

for i in range(1,11):
    fd2 = np.concatenate((fd2, dSet2[i]), axis=0)
    
t3 = d3[0:96]
d3a =  d3[96:]
fd3 = d3a
dSet3 = [d3a, d1, d2, d4, d5, d6, d7, d8, d9, d10, d11]

for i in range(1,11):
    fd3 = np.concatenate((fd3, dSet3[i]), axis=0)
    
t4 = d4[0:11]
d4a =  d4[11:]
fd4 = d4a
dSet4 = [d4a, d1, d2, d3, d5, d6, d7, d8, d9, d10, d11]

for i in range(1,11):
    fd4 = np.concatenate((fd4, dSet4[i]), axis=0)
    
t5 = d5[0:23]
d5a =  d5[23:]
fd5 = d5a
dSet5 = [d5a, d1, d2, d4, d5, d6, d7, d8, d9, d10, d11]

for i in range(1,11):
    fd5 = np.concatenate((fd5, dSet5[i]), axis=0)
    
    
    
for i in range(1,11):
    total=0
    for j in range(5):
        four, five, six, seven, eight = modelRunTest(fd1, t1, i*5, 32, 0)
        total+=four
    print(total/5, "% epochs: ", i*5)
    
    total=0
    for j in range(5):
        four, five, six, seven, eight = modelRunTest(fd2, t2, i*5, 32, 0)
        total+=four
    print(total/5, "% epochs: ", i*5)
    
    total=0
    for j in range(5):
        four, five, six, seven, eight = modelRunTest(fd3, t3, i*5, 32, 0)
        total+=four
    print(total/5, "% epochs: ", i*5)
    
    total=0
    for j in range(5):
        four, five, six, seven, eight = modelRunTest(fd4, t4, i*5, 32, 0)
        total+=four
    print(total/5, "% epochs: ", i*5)
    
    total=0
    for j in range(5):
        four, five, six, seven, eight = modelRunTest(fd5, t5, i*5, 32, 0)
        total+=four
    print(total/5, "% epochs: ", i*5)
    print("\n")


0.7101449275362319 % epochs:  5
0.6589285714285713 % epochs:  5
0.7020833333333333 % epochs:  5
0.8545454545454545 % epochs:  5
0.791304347826087 % epochs:  5


0.6608695652173913 % epochs:  10
0.6285714285714287 % epochs:  10
0.7208333333333332 % epochs:  10
0.6545454545454545 % epochs:  10
0.7652173913043477 % epochs:  10


0.672463768115942 % epochs:  15
0.7000000000000001 % epochs:  15
0.7333333333333333 % epochs:  15
0.6 % epochs:  15
0.6956521739130436 % epochs:  15


0.6521739130434783 % epochs:  20
0.7017857142857142 % epochs:  20


In [218]:
for j in range(1,11):   
    total = 0
    for i in range(10):
        one, two, three, four, five = modelRunTest(fd,t, 5*j, 32, 3) 
        total+=one
    print("Accuracy is ",total*10, "  % on ", 5*j, " epochs.")
     

Train on 7576 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
Train on 7576 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
Train on 7576 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
Train on 7576 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
Train on 7576 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
Train on 7576 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
Train on 7576 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
Train on 7576 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
Train on 7576 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
Train on 7576 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
Accuracy is  66.33928571428571   % on  5  epochs.
Train on 7576 samples
Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
Train on 7576 samples
Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/1

KeyboardInterrupt: 

Model and Training

Testing

In [95]:
# load dataset
p1 = "C:\\Users\\16479\\Desktop\\Astrobot Data\\Old Data\\savant_data (12).csv"
p2 = "C:\\Users\\16479\\Desktop\\Astrobot Data\\Old Data\\savant_data (5).csv"
p3 = "C:\\Users\\16479\\Desktop\\Astrobot Data\\Old Data\\savant_data (6).csv"
p4 = "C:\\Users\\16479\\Desktop\\Astrobot Data\\Old Data\\savant_data (7).csv"
p5 = "C:\\Users\\16479\\Desktop\\Astrobot Data\\Old Data\\savant_data (8).csv"
p6 = "C:\\Users\\16479\\Desktop\\Astrobot Data\\Old Data\\savant_data (9).csv"
p7 = "C:\\Users\\16479\\Desktop\\Astrobot Data\\Old Data\\savant_data (10).csv"
p8 = "C:\\Users\\16479\\Desktop\\Astrobot Data\\Old Data\\savant_data (11).csv"
p9 = "C:\\Users\\16479\\Desktop\\Astrobot Data\\Old Data\\savant_data (13).csv"
p10 = "C:\\Users\\16479\\Desktop\\Astrobot Data\\Old Data\\savant_data (14).csv"
p11 = "C:\\Users\\16479\\Desktop\\Astrobot Data\\Old Data\\savant_data (15).csv"

d1 = dataPipe3(p1)
d2 = dataPipe3(p2)
d3 = dataPipe3(p3)
d4 = dataPipe3(p4)
d5 = dataPipe3(p5)
d6 = dataPipe3(p6)
d7 = dataPipe3(p7)
d8 = dataPipe3(p8)
d9 = dataPipe3(p9)
d10 = dataPipe3(p10)
d11 = dataPipe3(p11)



In [91]:
dTrain = d3[96:]
dTest = d3[0:96]
j,i=0,0
for j in range(17,30):   
    total = 0
    for i in range(10):
        one, two, three, four, five = modelRunTest(dTrain, dTest, 5*j, 32, 0) 
        total+=one
    print("Accuracy is ",total*10, "  % on ", 5*j, " epochs.")

Accuracy is  78.125   % on  85  epochs.
Accuracy is  77.60416666666669   % on  90  epochs.
Accuracy is  83.125   % on  95  epochs.
Accuracy is  78.74999999999999   % on  100  epochs.
Accuracy is  79.68750000000001   % on  105  epochs.
Accuracy is  78.02083333333333   % on  110  epochs.
Accuracy is  79.89583333333333   % on  115  epochs.
Accuracy is  81.14583333333334   % on  120  epochs.
Accuracy is  81.45833333333331   % on  125  epochs.
Accuracy is  81.77083333333334   % on  130  epochs.
Accuracy is  81.25   % on  135  epochs.
Accuracy is  79.58333333333334   % on  140  epochs.
Accuracy is  78.54166666666667   % on  145  epochs.


In [101]:
accData=np.zeros((30,2))

for i in range(0,150, 5):
    e = i
    r = 5
    total = 0
    for i in range(r):
        one, two, three, four, five = modelRunTest(fd3, t3, e, 32, 0) 
        total+=one
    print("Accuracy: ", total/r*100, " % Global: ", e," epochs")
    accData[int(i/5),0]= total/r*100

    total = 0 
    for i in range(r):
        one, two, three, four, five = modelRunTest(d3a, t3, e, 32, 0) 
        total+=one
    print("Accuracy: ", total/r*100, " % Local: ", e, " epochs")
    accData[int(i/5),1]= total/r*100

    

Accuracy:  20.625  % Global:  0  epochs
Accuracy:  25.208333333333332  % Local:  0  epochs
Accuracy:  71.45833333333333  % Global:  5  epochs
Accuracy:  59.375  % Local:  5  epochs
Accuracy:  78.125  % Global:  10  epochs
Accuracy:  63.125  % Local:  10  epochs
Accuracy:  80.83333333333333  % Global:  15  epochs
Accuracy:  70.0  % Local:  15  epochs
Accuracy:  74.16666666666666  % Global:  20  epochs
Accuracy:  73.54166666666667  % Local:  20  epochs
Accuracy:  75.83333333333334  % Global:  25  epochs
Accuracy:  71.45833333333333  % Local:  25  epochs
Accuracy:  67.70833333333333  % Global:  30  epochs
Accuracy:  78.54166666666667  % Local:  30  epochs
Accuracy:  79.16666666666667  % Global:  35  epochs
Accuracy:  75.83333333333334  % Local:  35  epochs
Accuracy:  68.75  % Global:  40  epochs
Accuracy:  73.125  % Local:  40  epochs
Accuracy:  65.83333333333334  % Global:  45  epochs
Accuracy:  79.79166666666666  % Local:  45  epochs
Accuracy:  77.5  % Global:  50  epochs
Accuracy:  78.

In [59]:

a, yhat, y_test, probs = modelRun(d4, 70, 32, 0)
pitches_List = ["Fastball", "Breaking Ball", "Offspeed", "Other"]
n_class =4
# pitch by pitch testing, checking how the accuracy varies between different predicted and thrown pitches
sampleNum, correct = 0,0
print(pitches_List[0])
for j in range(n_class):
    print(pitches_List[j]," Testing Thrown")
    for i in range(len(y_test)):
        #print(y_test[i], yhat[i])
        if(y_test[i]==j):
            sampleNum+=1
            if y_test[i]==yhat[i]:
                correct+=1
    if(sampleNum !=0 or correct !=0):
        print(sampleNum, correct, correct/sampleNum*100, "%")
    else:
        print("Nan")

    sampleNum, correct = 0,0
    


for j in range(n_class):
    print(pitches_List[j]," Testing Predictions")
    for i in range(len(y_test)):
        #print(y_test[i], yhat[i])
        if(yhat[i]==j):
            sampleNum+=1
            if y_test[i]==yhat[i]:
                correct+=1
    if sampleNum !=0 and correct !=0:
        print(sampleNum, correct, correct/sampleNum*100, "%")
    else:
        print("NaN")

    sampleNum, correct = 0,0


ValueError: too many values to unpack (expected 4)

In [229]:
# Game day testing, no retraining
# split data into train and test sets
from sklearn import preprocessing
X = preprocessing.scale(X)
X_train, X_test, y_train, y_test = X[100:], X[0:100], y[100:], y[0:100]
#X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.2, random_state=2)

y = LabelEncoder().fit_transform(y)
n_class = len(unique(y))


n_features=62
model = Sequential()
model.add(Dense(90, input_dim=n_features, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(70, input_dim=n_features, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(10, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(6, activation='softmax'))


model.compile(loss='sparse_categorical_crossentropy', optimizer="SGD")
# fit the keras model on the dataset


history=model.fit(X_train, y_train, epochs=120, batch_size=16, verbose=0)


# evaluate on test set
yhat = model.predict(X_test)
probs = model.predict_proba(X_test)

yhat = argmax(yhat, axis=-1).astype('int')
acc = accuracy_score(y_test, yhat)
print('Accuracy: %.3f' % acc)

NameError: name 'X' is not defined

In [295]:
#Game day retraining, accuracy with retrains every ~ 10 pitches
n_features =53
model = Sequential()
model.add(Dense(90, input_dim=n_features, activation='relu', kernel_initializer='he_normal'))
#model.add(Dense(40, input_dim=n_features, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(10, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(6, activation='softmax'))


model.compile(loss='sparse_categorical_crossentropy', optimizer="SGD")
# fit the keras model on the dataset

bs = 32
e = 120
v = 0

X_train, X_test, y_train, y_test = X[113:], X[100:113], y[113:], y[100:113]
history=model.fit(X_train, y_train, epochs=e, batch_size=bs, verbose=v)
# evaluate on test set
yhat = model.predict(X_test)
yhat = argmax(yhat, axis=-1).astype('int')
acc = accuracy_score(y_test, yhat)
print('Accuracy: %.3f' % acc)

X_train, X_test, y_train, y_test = X[100:], X[90:100], y[100:], y[90:100]
history=model.fit(X_train, y_train, epochs=e, batch_size=bs, verbose=v)
# evaluate on test set
yhat = model.predict(X_test)
yhat = argmax(yhat, axis=-1).astype('int')
acc = accuracy_score(y_test, yhat)
print('Accuracy: %.3f' % acc)

X_train, X_test, y_train, y_test = X[90:], X[80:90], y[90:], y[80:90]
history=model.fit(X_train, y_train, epochs=e, batch_size=bs, verbose=v)
# evaluate on test set
yhat = model.predict(X_test)
yhat = argmax(yhat, axis=-1).astype('int')
acc = accuracy_score(y_test, yhat)
print('Accuracy: %.3f' % acc)

X_train, X_test, y_train, y_test = X[80:], X[70:80], y[80:], y[70:80]
history=model.fit(X_train, y_train, epochs=e, batch_size=bs, verbose=v)
# evaluate on test set
yhat = model.predict(X_test)
yhat = argmax(yhat, axis=-1).astype('int')
acc = accuracy_score(y_test, yhat)
print('Accuracy: %.3f' % acc)

X_train, X_test, y_train, y_test = X[70:], X[60:70], y[70:], y[60:70]
history=model.fit(X_train, y_train, epochs=e, batch_size=bs, verbose=v)
# evaluate on test set
yhat = model.predict(X_test)
yhat = argmax(yhat, axis=-1).astype('int')
acc = accuracy_score(y_test, yhat)
print('Accuracy: %.3f' % acc)

X_train, X_test, y_train, y_test = X[60:], X[50:60], y[60:], y[50:60]
history=model.fit(X_train, y_train, epochs=e, batch_size=bs, verbose=v)
# evaluate on test set
yhat = model.predict(X_test)
yhat = argmax(yhat, axis=-1).astype('int')
acc = accuracy_score(y_test, yhat)
print('Accuracy: %.3f' % acc)

X_train, X_test, y_train, y_test = X[50:], X[40:50], y[50:], y[40:50]
history=model.fit(X_train, y_train, epochs=e, batch_size=bs, verbose=v)
# evaluate on test set
yhat = model.predict(X_test)
yhat = argmax(yhat, axis=-1).astype('int')
acc = accuracy_score(y_test, yhat)
print('Accuracy: %.3f' % acc)

X_train, X_test, y_train, y_test = X[40:], X[30:40], y[40:], y[30:40]
history=model.fit(X_train, y_train, epochs=e, batch_size=bs, verbose=v)
# evaluate on test set
yhat = model.predict(X_test)
yhat = argmax(yhat, axis=-1).astype('int')
acc = accuracy_score(y_test, yhat)
print('Accuracy: %.3f' % acc)

X_train, X_test, y_train, y_test = X[30:], X[20:30], y[30:], y[20:30]
history=model.fit(X_train, y_train, epochs=e, batch_size=bs, verbose=v)
# evaluate on test set
yhat = model.predict(X_test)
yhat = argmax(yhat, axis=-1).astype('int')
acc = accuracy_score(y_test, yhat)
print('Accuracy: %.3f' % acc)

X_train, X_test, y_train, y_test = X[20:], X[10:20], y[20:], y[10:20]
history=model.fit(X_train, y_train, epochs=e, batch_size=bs, verbose=v)
# evaluate on test set
yhat = model.predict(X_test)
yhat = argmax(yhat, axis=-1).astype('int')
acc = accuracy_score(y_test, yhat)
print('Accuracy: %.3f' % acc)

X_train, X_test, y_train, y_test = X[10:], X[0:10], y[10:], y[0:10]
history=model.fit(X_train, y_train, epochs=e, batch_size=bs, verbose=v)
# evaluate on test set
yhat = model.predict(X_test)
yhat = argmax(yhat, axis=-1).astype('int')
acc = accuracy_score(y_test, yhat)
print('Accuracy: %.3f' % acc)



Accuracy: 0.692
Accuracy: 0.500
Accuracy: 0.600
Accuracy: 0.200
Accuracy: 0.500
Accuracy: 0.500
Accuracy: 0.300
Accuracy: 0.200
Accuracy: 0.200
Accuracy: 0.700
Accuracy: 0.200


Extras

In [ ]:
import numpy as np

innings = np.zeros((9,4))
for j in range(len(y)):
    innings[int(X[j,7])-1, int(y[j])]+=1
print(innings)

inningsPercent = innings
for i in range(len(innings)):
    inningsPercent[i]=innings[i]/np.sum(innings[i])
print(inningsPercent)

strikes = np.zeros((3,4))
for j in range(len(y)):
    strikes[int(X[j,2]), int(y[j])]+=1

print(strikes)

strikesPercent = strikes
for i in range(len(strikes)):
    strikesPercent[i]=strikes[i]/np.sum(strikes[i])
print(strikesPercent)

balls = np.zeros((4,4))
for j in range(len(y)):
    balls[int(X[j,1]), int(y[j])]+=1

print(balls)

ballsPercent = balls
for i in range(len(balls)):
    ballsPercent[i]=balls[i]/np.sum(balls[i])
print(ballsPercent)

pitchNum = np.zeros((12,4))
for j in range(len(y)):
    pitchNum[int(X[j,8]), int(y[j])]+=1

print(pitchNum)

pitchNumPercent = pitchNum
for i in range(len(pitchNum)):
    if( 0!=np.sum(pitchNum[i])):
        pitchNumPercent[i]=pitchNum[i]/np.sum(pitchNum[i])
print(pitchNumPercent)

In [ ]:
# evaluate the model
train_acc = model.evaluate(X_train, y_train, verbose=0)
test_acc = model.evaluate(X_test, y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
# plot loss during training
pyplot.subplot(211)
pyplot.title('Loss')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
# plot accuracy during training
pyplot.subplot(212)
pyplot.title('Accuracy')
pyplot.plot(history.history['accuracy'], label='train')
pyplot.plot(history.history['val_accuracy'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
for i in range(10):
    #l = 32
    data_X, data_y = b[:, 1:-1], b[:, -1]
    data_X, data_y = data_X.astype('float'), data_y.astype('float')
    n_features = data_X.shape[1]
    data_X = preprocessing.scale(data_X)
    #X_train, X_test, y_train, y_test = data_X[l:], data_X[0:l], data_y[l:], data_y[0:l]
    X_train, X_test, y_train, y_test =train_test_split(data_X, data_y, test_size=0.2, random_state=2)

 d1a, d2a, d3a, d4a, d5a, d6a = 0, 0, 0, 0, 0, 0

da = [d1a, d2a, d3a, d4a, d5a, d6a]
for i in range(6):
    x,y = np.shape(d[i])
    da[i] = np.zeros((x,y-2))
    da[i][:,0:14] = d[i][:,0:14]
    da[i][:,14:] = d[i][:,16:]